In [ ]:
-- Unidades, ingresos y margen por día
SELECT fecha,
       SUM(cantidad) AS unidades,
       SUM(cantidad*p.precio_unitario*(1 - descuento_pct/100.0)) AS ingresos,
       SUM(cantidad*(p.precio_unitario-p.costo_unitario)) AS margen
FROM dbo.fact_ventas fv
JOIN dbo.dim_productos p ON fv.producto_id = p.producto_id
GROUP BY fecha;